In [ ]:
import joblib
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score, f1_score, classification_report


In [ ]:
test_df = pd.read_parquet("data/test.parquet")
test_df = test_df.dropna(subset=["text", "label"]).copy()

def clean_text(text):
    import re
    if not isinstance(text, str):
        return ""
    text = text.strip()
    text = re.sub(r"\s+", " ", text)
    return text

X_test = test_df["text"].astype(str).map(clean_text)
y_test = test_df["label"]

print("Test samples:", len(X_test))
print("Labels:", y_test.unique())


In [ ]:
baseline_model = joblib.load("models/tfidf_logreg.joblib")
word_char_model = joblib.load("models/tfidf_word_char_logreg.joblib")

models = {
    "TF-IDF (Word only)": baseline_model,
    "TF-IDF (Word + Char)": word_char_model
}


In [ ]:
results = []

for name, model in models.items():
    y_pred = model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1_macro = f1_score(y_test, y_pred, average="macro")
    f1_weighted = f1_score(y_test, y_pred, average="weighted")

    results.append({
        "Model": name,
        "Accuracy": acc,
        "Macro F1": f1_macro,
        "Weighted F1": f1_weighted
    })

    print(f"\n=== {name} ===")
    print("Accuracy:", acc)
    print("Macro F1:", f1_macro)
    print("Weighted F1:", f1_weighted)


In [ ]:
results_df = pd.DataFrame(results)
results_df


In [ ]:
print("\n=== Class-wise performance: Baseline ===")
print(classification_report(y_test, baseline_model.predict(X_test), digits=4))

print("\n=== Class-wise performance: Word + Char ===")
print(classification_report(y_test, word_char_model.predict(X_test), digits=4))
